In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Import and clean data

In [184]:
# import data
data = pd.read_csv('us_acs_qf_clean_density.csv')

# clean data
# remove index column since it doesn't provide valuable info
data = data.drop(["place_id"], axis = 1)
# remove state_id
data = data.drop(["state_id"], axis = 1)

# REMOVE NULL / BAD DATA

# remove city names for scaling
locations = data["Name"]
data = data.drop(["Name"], axis=1)

# standardize data
scaler = StandardScaler()
scaler.fit(data)
scaled_data = pd.DataFrame(scaler.transform(data), columns = data.columns)

# Add weights to model

In [188]:
no_weights = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0]
no_weights = pd.Series(no_weights)
data_no_weights = scaled_data.copy()

i = 0
for column in data_no_weights:
    data_no_weights[column] = data_no_weights[column].multiply(no_weights[i])
    i = i + 1 

data_no_weights.head()

,Total Population,Density,Land Area,Median Household Income,Per capita income,Under 5yo %,Under 18yo %,Over 65yo %,Female %,White %,...,Households with internet %,High school graduate or higher %,Bachelor's degree or higher %,18-64yo with disability %,Without health insurance %,In labor force over 16yo %,In labor force females over 16yo %,People in poverty %,Latitude,Longitude
0,-0.112362,-0.698660,0.198877,-0.897897,-0.565466,-0.955067,-1.052584,-0.085594,-0.601737,-1.367759,...,0.066771,-1.402390,-0.443134,0.960143,-0.154871,-2.085416,-2.490406,0.684580,-1.381490,0.403246
1,-0.086106,-0.687551,0.461011,-0.600046,-0.722889,-0.157268,-0.134881,-0.067873,1.070031,-2.086376,...,-1.520285,-1.473490,-0.670283,1.203098,0.372076,-1.031811,-0.851988,1.371425,-0.992881,0.288882
2,-0.131096,-0.679090,-0.090400,-0.669607,-0.468332,1.238880,0.220359,-0.138496,-0.009212,0.762966,...,-0.019796,-0.833587,-0.891453,-0.376112,0.101075,0.505100,0.198047,-0.366502,-0.878380,0.275092
3,0.265015,-0.150839,0.440084,0.598056,0.212832,0.175148,0.457186,0.200440,0.329374,-0.503408,...,0.999767,0.493619,0.836069,-0.443599,0.176353,0.833748,0.686857,-0.741145,-1.061143,0.297173
4,0.132331,-0.376854,0.478795,-0.419899,-0.539087,0.640531,1.286079,0.117456,0.414021,0.345866,...,0.595789,-1.248339,-0.263807,-0.362614,1.064634,-0.229524,-0.336022,0.726207,-0.866851,0.339697


In [189]:
pop_weights = [2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0, 1.0]
pop_weights = pd.Series(pop_weights)
data_pop_weights = scaled_data.copy()

i = 0
for column in data_pop_weights:
    data_pop_weights[column] = data_pop_weights[column].multiply(pop_weights[i])
    i = i + 1 

data_pop_weights.head()

,Total Population,Density,Land Area,Median Household Income,Per capita income,Under 5yo %,Under 18yo %,Over 65yo %,Female %,White %,...,Households with internet %,High school graduate or higher %,Bachelor's degree or higher %,18-64yo with disability %,Without health insurance %,In labor force over 16yo %,In labor force females over 16yo %,People in poverty %,Latitude,Longitude
0,-0.224725,-0.698660,0.198877,-0.897897,-0.565466,-0.955067,-1.052584,-0.085594,-0.601737,-1.367759,...,0.066771,-1.402390,-0.443134,0.960143,-0.154871,-2.085416,-2.490406,0.684580,-1.381490,0.403246
1,-0.172213,-0.687551,0.461011,-0.600046,-0.722889,-0.157268,-0.134881,-0.067873,1.070031,-2.086376,...,-1.520285,-1.473490,-0.670283,1.203098,0.372076,-1.031811,-0.851988,1.371425,-0.992881,0.288882
2,-0.262191,-0.679090,-0.090400,-0.669607,-0.468332,1.238880,0.220359,-0.138496,-0.009212,0.762966,...,-0.019796,-0.833587,-0.891453,-0.376112,0.101075,0.505100,0.198047,-0.366502,-0.878380,0.275092
3,0.530029,-0.150839,0.440084,0.598056,0.212832,0.175148,0.457186,0.200440,0.329374,-0.503408,...,0.999767,0.493619,0.836069,-0.443599,0.176353,0.833748,0.686857,-0.741145,-1.061143,0.297173
4,0.264661,-0.376854,0.478795,-0.419899,-0.539087,0.640531,1.286079,0.117456,0.414021,0.345866,...,0.595789,-1.248339,-0.263807,-0.362614,1.064634,-0.229524,-0.336022,0.726207,-0.866851,0.339697


In [239]:
location_weights = [0.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 2.0, 2.0]
location_weights = pd.Series(location_weights)
data_location_weights = scaled_data.copy()

i = 0
for column in data_location_weights:
    data_location_weights[column] = data_location_weights[column].multiply(location_weights[i])
    i = i + 1 

data_location_weights.head()

,Total Population,Density,Land Area,Median Household Income,Per capita income,Under 5yo %,Under 18yo %,Over 65yo %,Female %,White %,...,Households with internet %,High school graduate or higher %,Bachelor's degree or higher %,18-64yo with disability %,Without health insurance %,In labor force over 16yo %,In labor force females over 16yo %,People in poverty %,Latitude,Longitude
0,-0.0,-0.349330,0.198877,-0.897897,-0.565466,-0.955067,-1.052584,-0.085594,-0.601737,-1.367759,...,0.066771,-1.402390,-0.443134,0.960143,-0.154871,-2.085416,-2.490406,0.684580,-2.762981,0.806492
1,-0.0,-0.343775,0.461011,-0.600046,-0.722889,-0.157268,-0.134881,-0.067873,1.070031,-2.086376,...,-1.520285,-1.473490,-0.670283,1.203098,0.372076,-1.031811,-0.851988,1.371425,-1.985763,0.577763
2,-0.0,-0.339545,-0.090400,-0.669607,-0.468332,1.238880,0.220359,-0.138496,-0.009212,0.762966,...,-0.019796,-0.833587,-0.891453,-0.376112,0.101075,0.505100,0.198047,-0.366502,-1.756759,0.550184
3,0.0,-0.075420,0.440084,0.598056,0.212832,0.175148,0.457186,0.200440,0.329374,-0.503408,...,0.999767,0.493619,0.836069,-0.443599,0.176353,0.833748,0.686857,-0.741145,-2.122286,0.594347
4,0.0,-0.188427,0.478795,-0.419899,-0.539087,0.640531,1.286079,0.117456,0.414021,0.345866,...,0.595789,-1.248339,-0.263807,-0.362614,1.064634,-0.229524,-0.336022,0.726207,-1.733702,0.679394


In [243]:
distance_weights = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
distance_weights = pd.Series(distance_weights)
data_distance_weights = scaled_data.copy()

i = 0
for column in data_distance_weights:
    data_distance_weights[column] = data_distance_weights[column].multiply(distance_weights[i])
    i = i + 1 

data_distance_weights.head()

,Total Population,Density,Land Area,Median Household Income,Per capita income,Under 5yo %,Under 18yo %,Over 65yo %,Female %,White %,...,Households with internet %,High school graduate or higher %,Bachelor's degree or higher %,18-64yo with disability %,Without health insurance %,In labor force over 16yo %,In labor force females over 16yo %,People in poverty %,Latitude,Longitude
0,-0.112362,-0.698660,0.198877,-0.897897,-0.565466,-0.955067,-1.052584,-0.085594,-0.601737,-1.367759,...,0.066771,-1.402390,-0.443134,0.960143,-0.154871,-2.085416,-2.490406,0.684580,-0.0,0.0
1,-0.086106,-0.687551,0.461011,-0.600046,-0.722889,-0.157268,-0.134881,-0.067873,1.070031,-2.086376,...,-1.520285,-1.473490,-0.670283,1.203098,0.372076,-1.031811,-0.851988,1.371425,-0.0,0.0
2,-0.131096,-0.679090,-0.090400,-0.669607,-0.468332,1.238880,0.220359,-0.138496,-0.009212,0.762966,...,-0.019796,-0.833587,-0.891453,-0.376112,0.101075,0.505100,0.198047,-0.366502,-0.0,0.0
3,0.265015,-0.150839,0.440084,0.598056,0.212832,0.175148,0.457186,0.200440,0.329374,-0.503408,...,0.999767,0.493619,0.836069,-0.443599,0.176353,0.833748,0.686857,-0.741145,-0.0,0.0
4,0.132331,-0.376854,0.478795,-0.419899,-0.539087,0.640531,1.286079,0.117456,0.414021,0.345866,...,0.595789,-1.248339,-0.263807,-0.362614,1.064634,-0.229524,-0.336022,0.726207,-0.0,0.0


In [295]:
age_weights = [1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 1.0, 1.0,
                    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
age_weights = pd.Series(age_weights)
data_age_weights = scaled_data.copy()

i = 0
for column in data_age_weights:
    data_age_weights[column] = data_age_weights[column].multiply(age_weights[i])
    i = i + 1 

data_age_weights.head()

,Total Population,Density,Land Area,Median Household Income,Per capita income,Under 5yo %,Under 18yo %,Over 65yo %,Female %,White %,...,Households with internet %,High school graduate or higher %,Bachelor's degree or higher %,18-64yo with disability %,Without health insurance %,In labor force over 16yo %,In labor force females over 16yo %,People in poverty %,Latitude,Longitude
0,-0.112362,-0.698660,0.198877,-0.897897,-0.565466,-2.865200,-3.157751,-0.256781,-0.601737,-1.367759,...,0.066771,-1.402390,-0.443134,0.960143,-0.154871,-2.085416,-2.490406,0.684580,-0.0,0.0
1,-0.086106,-0.687551,0.461011,-0.600046,-0.722889,-0.471804,-0.404642,-0.203620,1.070031,-2.086376,...,-1.520285,-1.473490,-0.670283,1.203098,0.372076,-1.031811,-0.851988,1.371425,-0.0,0.0
2,-0.131096,-0.679090,-0.090400,-0.669607,-0.468332,3.716640,0.661078,-0.415487,-0.009212,0.762966,...,-0.019796,-0.833587,-0.891453,-0.376112,0.101075,0.505100,0.198047,-0.366502,-0.0,0.0
3,0.265015,-0.150839,0.440084,0.598056,0.212832,0.525445,1.371557,0.601319,0.329374,-0.503408,...,0.999767,0.493619,0.836069,-0.443599,0.176353,0.833748,0.686857,-0.741145,-0.0,0.0
4,0.132331,-0.376854,0.478795,-0.419899,-0.539087,1.921593,3.858236,0.352369,0.414021,0.345866,...,0.595789,-1.248339,-0.263807,-0.362614,1.064634,-0.229524,-0.336022,0.726207,-0.0,0.0


In [273]:
scaled_data.columns
#5, 6, 7

Index(['Total Population', 'Density', 'Land Area', 'Median Household Income',
       'Per capita income', 'Under 5yo %', 'Under 18yo %', 'Over 65yo %',
       'Female %', 'White %', 'Black or African American %',
       'American Indian and Alaska Native %', 'Asian %',
       'Native Hawaiian and Other Pacific Islander %', 'Two or more races %',
       'Hispanic or Latino %', 'White alone, not Hispanic or Latino %',
       'Foreign born %', 'Total Housing Units', 'Owner-occupied Housing Units',
       'Median value of owner-occupied housing units',
       'Median selected monthly owner costs - with mortgage',
       'Median selected monthly owner costs - without mortgage',
       'Median gross rent', 'Total households', 'Persons per household',
       'Living in the same house 1 year ago %',
       'Language other than English spoken at home %',
       'Households with computer %', 'Households with internet %',
       'High school graduate or higher %', 'Bachelor's degree or higher %',

## Categories
- No weights (data_no_weights)
- Population (data_pop_weights)
- Location
- Distance (ie. far away)
- Demographics (age)

- Demographics (race)
- Housing
- Education
- Economics
- Smart score (everything)

# Run KNN algorithm

In [296]:
nbrs = NearestNeighbors(n_neighbors=20,
                        algorithm="ball_tree").fit(data_distance_weights)
distances, indices = nbrs.kneighbors(data_distance_weights)

# Query a city

In [196]:
query = locations.str.contains("milwaukee(?i)")
query.index[query]

Int64Index([20968, 21140, 21218], dtype='int64')

In [293]:
locations[15374]

'The Village city, Oklahoma'

In [287]:
for i in indices[4634]:
    print(locations[i])

Evanston city, Illinois
Oak Park village, Illinois
Medford city, Massachusetts
Watertown Town city, Massachusetts
Shorewood village, Wisconsin
Shoreline city, Washington
Highland Park borough, New Jersey
Wheaton city, Illinois
Towson CDP, Maryland
Waltham city, Massachusetts
Arlington Heights village, Illinois
Arlington CDP, Massachusetts
Winthrop Town city, Massachusetts
Beaverton city, Oregon
Annapolis city, Maryland
Ardmore CDP, Pennsylvania
Edmonds city, Washington
Red Bank borough, New Jersey
Drexel Hill CDP, Pennsylvania
Alexandria city, Virginia


In [288]:
for i in indices[20968]:
    print(locations[i])

Milwaukee city, Wisconsin
Baltimore city, Maryland
Atlanta city, Georgia
Omaha city, Nebraska
Cleveland city, Ohio
Raleigh city, North Carolina
Sacramento city, California
Fresno city, California
Las Vegas city, Nevada
Mesa city, Arizona
New Orleans city, Louisiana
Tulsa city, Oklahoma
Minneapolis city, Minnesota
Wichita city, Kansas
Detroit city, Michigan
Tampa city, Florida
Tucson city, Arizona
Colorado Springs city, Colorado
St. Louis city, Missouri
Memphis city, Tennessee


In [256]:
for i in indices[2558]:
    print(locations[i])

Norwalk city, Connecticut
Framingham city, Massachusetts
Stamford city, Connecticut
Danbury city, Connecticut
White Plains city, New York
Plantation city, Florida
Palatine village, Illinois
Davie town, Florida
Marlborough city, Massachusetts
Mount Prospect village, Illinois


In [289]:
for i in indices[4492]:
    print(locations[i])

Chicago city, Illinois
Houston city, Texas
Los Angeles city, California
Philadelphia city, Pennsylvania
Phoenix city, Arizona
San Antonio city, Texas
San Diego city, California
Dallas city, Texas
Austin city, Texas
Jacksonville city, Florida
Indianapolis city (balance), Indiana
San Jose city, California
Columbus city, Ohio
Charlotte city, North Carolina
San Francisco city, California
Fort Worth city, Texas
Seattle city, Washington
Denver city, Colorado
Washington city, District of Columbia
Nashville-Davidson metropolitan government (balance), Tennessee


In [290]:
for i in indices[20936]:
    print(locations[i])

Madison city, Wisconsin
Lincoln city, Nebraska
Boise City city, Idaho
Overland Park city, Kansas
Reno city, Nevada
Spokane city, Washington
Salt Lake City city, Utah
Tacoma city, Washington
Chandler city, Arizona
St. Petersburg city, Florida
Eugene city, Oregon
Pittsburgh city, Pennsylvania
Durham city, North Carolina
Sioux Falls city, South Dakota
Fort Collins city, Colorado
Des Moines city, Iowa
St. Paul city, Minnesota
Lakewood city, Colorado
Vancouver city, Washington
Tempe city, Arizona


In [297]:
for i in indices[3377]:
    print(locations[i])

The Villages CDP, Florida
Sun City West CDP, Arizona
Sun City CDP, Arizona
Green Valley CDP, Arizona
Sun City Center CDP, Florida
Sun Lakes CDP, Arizona
On Top of the World CDP, Florida
Venice city, Florida
Hot Springs Village CDP, Arkansas
Englewood CDP, Florida
Saddlebrooke CDP, Arizona
Estero village, Florida
Fairfield Glade CDP, Tennessee
Prescott city, Arizona
Iona CDP, Florida
Gold Canyon CDP, Arizona
Desert Palms CDP, California
Plantation CDP, Florida
Timber Pines CDP, Florida
Punta Gorda city, Florida


In [291]:
query = locations.str.contains("the villa(?i)")
query.index[query]

Int64Index([3377, 15374], dtype='int64')

# Export to JSON file

In [60]:
import json

In [63]:
indices

array([[    0,   112,    61, ..., 10238,   787,  3474],
       [    1,   215,  3580, ...,  7715,  3952,  3695],
       [    2,  4429,   199, ...,  8756,  6749,   881],
       ...,
       [21332, 20062, 19224, ...,  8505, 12433, 20143],
       [21333, 21272, 21185, ..., 14870, 18599, 17588],
       [21334, 19006, 18512, ..., 13341, 17354, 19246]], dtype=int64)

In [64]:
with open("100nn_cities.json", 'w') as file:
    json.dump(indices.tolist(), file)